# Age-association analyses of closely-related HA sequence pairs

## Overview 
In this notebook, we demonstrate the analyses performed in our manuscript entitiled, "Individual immune selection pressure has limited impact on seasonal influenza virus evolution". As an example, we show how we analysed for the hemagglutinin (HA) sequences of A/H3N2 collected between 2009 and July-2016.

## Pre-analysis tasks 
### Sequence curation
We collected all sequences from [GISAID](http://www.gisaid.org/) and [NCBI Genbank](http://www.ncbi.nlm.nih.gov/genbank/) databases that satisfy the following filters: 
  1. \>90% of full HA nucleotide length 
  2. availability of patients' age data 
  3. availability of virus passage history that could be categorized as original clinical material, MDCK, SIAT and egg passage types (Typing of passage histories was based on [Chan et al., 2016](https://www.ncbi.nlm.nih.gov/pubmed/27604224)). 

In total, 10,514 HA nucleotide sequences were obtained for A/H3N2.

We further processed the sequences obtained by: 
  1. Removed duplicated and low quality (\>10% residues missing or ambiguous) were removed. 
  2. Sequences of the same virus but different passage histories were prioritized by clinical > MDCK > SIAT > egg and/or lower passage number. **At this point, there are 8,530 sequences for A/H3N2** (i.e. H3N2-HA-nuc.fa in [Files](https://github.com/alvinxhan/ageflu/tree/master/files) folder). 
  3. Clustered and removed identical nucleotide sequences using CD-HIT, leaving behind a representative strain for each cluster to yield a final set of non-redundant sequences. You can find the CD-HIT sequence cluster output file (e.g. H3N2-HA-nuc_cd-hit-non-redundant.clstr) in the [Files](https://github.com/alvinxhan/ageflu/tree/master/files) folder. 
  
In total, there were 6,033 non-redudant sequences for A/H3N2. 

### Maximum-likelihood tree reconstruction
Prior to the analyses presented here, we first reconstruct a maximum-likelihood (ML) phylogenetic tree using HA nucleotide sequences. Due to the large number of sequences (n=6,033 for A/H3N2) and the relatively low observed genetic divergence (overall mean nucleotide p-distance of A/H3N2 calculated by MEGA = 0.00964), conventional phylogenetic methods would be computationally expensive and practically infeasible. As such, we reconstructed our ML trees with RAxML and GARLI, using a nested inference approach that we have developed. More information on this phylogenetic inference methodology can be found in our paper. 

The phylogenetic tree for A/H3N2 (H3N2-HA-nuc.rooted.nwk) can be found in the [Files](https://github.com/alvinxhan/ageflu/tree/master/files) folder as well. 

## Getting evolutionarily closest pairs
We now can parse for evolutionarily closely-related virus pairs from the inferred phylogenetic tree. You can do so using [ageflu_getpairs.py](https://github.com/alvinxhan/ageflu/blob/master/scripts/ageflu_getpairs.py). 

* Additional inputs include: (i) FASTA alignment of all sequences (including identical nucleotide sequences), (ii) CD-HIT cluster file of non-redundant sequence clusters.
* The age limits of children and adults can be changed using the '--child' and '--adult' arguments.
* Note that the FASTA alignment containing reference sequences for HA-numbering ('H1pdm09_H3_FluB_NumberingRef.fa') must be in the same folder as the ageflu_getpairs.py script.
* Tab-delimited output: ageflu_evol-closest-pairs.*.txt

Here, we show the breakdown of the script:

### Importing modules and defining functions

In [1]:
from os.path import expanduser
from decimal import *
import re, sys, itertools
import ete3
import random

random.seed(666)

# parse fasta file
def parsefasta(file, check_alg=1, check_dna=1, check_codon=1):
    # check if file is of FASTA format
    fhandle = filter(None, open(file, 'rU').readlines())
    if not re.search('^>', fhandle[0]): sys.exit('\nERROR: Incorrect sequence file format.\n')

    result = {}
    for key, group in itertools.groupby(fhandle, lambda _: re.search('^>',_)):
        if key:
            header = group.next().strip().replace('>','')
        else:
            sequence = ''.join(map(lambda _:_.strip(),list(group)))
            # dna sequences
            if check_dna == 1 and set(list(sequence))&set(list('rhkdesqpvilmfyw')):
                sys.exit('\nERROR: Input must be DNA FASTA.\n')
            # codon-alignment
            if check_codon == 1 and len(sequence)%3 > 0:
                sys.exit('\nERROR: DNA sequence file given must be codon-aligned.\n')
            result[header] = sequence
    # alignment
    if check_alg == 1 and len(set(map(len, result.values()))) != 1:
        sys.exit('\nERROR: Input sequence file must be an alignment.\n')
    return result

# translate codon-aligned nucleotide sequences
def translatedDNA(dna):
    protein = []
    for _ in xrange(0,len(dna),3):
        codon = dna[_:_+3]
        if codon in dnacodontable:
            if dnacodontable[codon] == 'stop': break
            protein.append(dnacodontable[codon])
        elif re.match('---',codon): protein.append('-')
        else: protein.append('X')
    return ''.join(protein)

# get pairwise substitutions
def get_pairwise_substitutions(anc_seq, desc_seq):
    # nucleotide sequences input
    if set(list(anc_seq)) <= set(list('atgcn-')):
        transitions, transversions, nonsyn_sub, syn_sub = 0, 0, 0, 0
        for _ in xrange(0, len(anc_seq), 3):
            anc_codon = anc_seq[_:_+3]
            desc_codon = desc_seq[_:_+3]
            if anc_codon == desc_codon:
                continue
            else:
                unknown_res_binary = 0
                nuc_sub = 0
                for j in xrange(3):
                    if re.match('(n|-)', anc_codon[j]) or re.match('(n|-)', desc_codon[j]):
                        unknown_res_binary = 1
                        continue
                    elif anc_codon[j] != desc_codon[j]:
                        if re.search('(ag|ga|ct|tc)', ''.join([anc_codon[j], desc_codon[j]])):
                            transitions += 1
                            nuc_sub += 1
                        else:
                            transversions += 1
                            nuc_sub += 1

                if unknown_res_binary == 1:
                    continue
                if dnacodontable[anc_codon] != dnacodontable[desc_codon]:
                    nonsyn_sub += nuc_sub
                else:
                    syn_sub += nuc_sub
        return {'transitions':transitions, 'transversions':transversions, 'nonsyn':nonsyn_sub, 'syn':syn_sub}
    # protein sequences input
    else:
        mutation_list = []
        for _ in xrange(len(anc_seq)):
            if re.search('(-|X)', anc_seq[_]) or re.search('(-|X)', desc_seq[_]):
                continue
            elif anc_seq[_] != desc_seq[_]:
                mutation_list.append('{}{}{}'.format(anc_seq[_], _+1, desc_seq[_]))
        return mutation_list

def get_least_unknown_residue_sequences(seq_dict, dist_dict_to_edit={}):
    seqid_to_unknownrescount = {seqid:len(re.findall('(-|n|X)', sequence, re.I)) for seqid, sequence in seq_dict.items()}
    seqid_with_least_unknown_res = [seqid for seqid, count in seqid_to_unknownrescount.items() if count == min(seqid_to_unknownrescount.values())]
    if len(dist_dict_to_edit) > 0:
        for seqid in list(set(seq_dict.keys())-set(seqid_with_least_unknown_res)):
            del dist_dict_to_edit[seqid]
        return seqid_with_least_unknown_res, dist_dict_to_edit
    else:
        return seqid_with_least_unknown_res

### Define inputs 

You can choose to analyse other influenza subtypes/lineages by changing ```subtype_to_analyse``` here. You can also opt to analyse only closely-related viruses of the same passage cell type by changing ```passage_requirement = 'T'```

In [2]:
subtype_to_analyse = 'H3N2'
passage_requirement = 'F'

subtype_to_passage_to_age_range = {'T':{'H3N2':{'child':[0, 12], 'adult':[25, 120]},
                                        'H1N1pdm09':{'child':[0, 5], 'adult':[20, 120]},
                                        'BVic':{'child':[0, 10], 'adult':[35, 120]},
                                        'BYam':{'child':[0, 10], 'adult':[35, 120]}},
                                   'F':{'H3N2':{'child':[0, 5], 'adult':[35, 120]},
                                        'H1N1pdm09':{'child':[0, 5], 'adult':[25, 120]},
                                        'BVic':{'child':[0, 10], 'adult':[20, 120]},
                                        'BYam':{'child':[0, 5], 'adult':[35, 120]}}}

class params:
    tree = './files/{st}/{st}-HA-nuc_rooted.nwk'.format(st=subtype_to_analyse) # Phylogenetic tree file in NEWICK format 
    aln = './files/{st}/{st}-HA-nuc.fa'.format(st=subtype_to_analyse) # Nucleotide alignment of HA sequences (pre-CD-HIT)
    nr = './files/{st}/{st}-HA-nuc_cd-hit-non-redundant.clstr'.format(st=subtype_to_analyse) # CD-HIT identical sequence cluster file 
    child = subtype_to_passage_to_age_range[passage_requirement][subtype_to_analyse]['child'] # age range of children 
    adult = subtype_to_passage_to_age_range[passage_requirement][subtype_to_analyse]['adult'] # age range of adults 
    maxmut = 5 # maxmimum number of amino acid substitutions allowable in an evolutionarily closest pair 
    patdist = 0.007 # maximum patristic distance between sequences in a closest pair
    order = 3 # max ordinal of pairs 

### Parse FASTA and CD-HIT files 
We sort CD-HIT identical sequence clusters into age categories ([C]hild, [A]dult or [U]ncateogrized) based on input children and adults age ranges. 

Any sequences in the an [U]ncategorized cluster will **NOT** be used for pairing in the subsequent analyses.

In [3]:
# query subtype analysed 
try:
    query_subtype = re.search('(H3N2|pH1N1|H1N1pdm09|BVic|BYam)', params.tree).group().upper()
    if query_subtype == 'PH1N1':
        query_subtype = 'H1N1PDM09'
except:
    query_subtype = raw_input('\nWARNING: Can\'t parsed query subtype from tree name. Enter subtype (H3N2/pH1N1/BVic/BYam): ')

# output filename
outfname = '{}C{}_{}A{}_PD{}_MM{}_CL{}_{}'.format(params.child[0], params.child[-1], params.adult[0], params.adult[-1], params.patdist, params.maxmut, params.order, re.sub('[^/]*/', '', params.tree))

# parse fasta alignment 
dnacodontable = {'ttt':'F', 'ttc':'F', 'tta':'L', 'ttg':'L', 'ctt':'L', 'ctc':'L', 'cta':'L', 'ctg':'L', 'att':'I', 'atc':'I', 'ata':'I', 'atg':'M', 'gtt':'V', 'gtc':'V', 'gta':'V', 'gtg':'V', 'tct':'S', 'tcc':'S', 'tca':'S', 'tcg':'S', 'cct':'P', 'ccc':'P', 'cca':'P', 'ccg':'P', 'act':'T', 'acc':'T', 'aca':'T', 'acg':'T', 'gct':'A', 'gcc':'A', 'gca':'A', 'gcg':'A', 'tat':'Y', 'tac':'Y', 'taa':'stop', 'tag':'stop', 'cat':'H', 'cac':'H', 'caa':'Q', 'cag':'Q', 'aat':'N', 'aac':'N', 'aaa':'K', 'aag':'K', 'gat':'D', 'gac':'D', 'gaa':'E', 'gag':'E', 'tgt':'C', 'tgc':'C', 'tga':'stop', 'tgg':'W', 'cgt':'R', 'cgc':'R', 'cga':'R', 'cgg':'R', 'agt':'S', 'agc':'S', 'aga':'R', 'agg':'R', 'ggt':'G', 'ggc':'G', 'gga':'G', 'ggg':'G'}

fdat_nuc = parsefasta(params.aln)
fdat_aa = {k:translatedDNA(v) for k,v in fdat_nuc.items()}
isolateid_to_fdatheader = {re.search('(GBISL|EPIISL|GB_ISL_|EPI_ISL_)\d+', header).group().replace('_', ''):header for header in fdat_nuc.keys()}
fdatheader_to_isolateid = {v:k for k,v in isolateid_to_fdatheader.items()}
isolateid_to_age = {isolateid:int(re.search('AGE(\d+)', header).group(1)) for isolateid, header in isolateid_to_fdatheader.items()}

# sequences to ignore (outside of child_min and adult_max age)
isolates_to_ignore = [isolateid for isolateid, age in isolateid_to_age.items() if age < params.child[0] or age > params.adult[-1]]

# parse clstr file and sort each identical sequence cluster (>1 member sequence) into children/adult age categories 
# isolates_to_ignore now include those of ambiguous age categories (between child_max and adult_min) 
# + 0 pat dist sequences with > minimal number of n/-
isolateid_to_representatives, isolateid_to_agecategory = {}, {} 
fhandle = filter(None, open(params.nr, 'rU').readlines())
for key, group in itertools.groupby(fhandle, lambda _: re.search('^>Cluster', _)):
    if not key:
        cluster = list(group)
        if len(cluster) > 1:
            cluster = [re.search('(GBISL|EPIISL|GB_ISL_|EPI_ISL_)\d+', header).group().replace('_', '') for header in cluster]
            cluster_age = [isolateid_to_age[isolateid] for isolateid in cluster]
            # all children or adults sequence clusters
            if all([params.child[0] <= age <= params.child[-1] for age in cluster_age]):
                isolateid_to_agecategory[isolateid] = 'C'
                for isolateid in cluster:
                    isolateid_to_representatives[isolateid] = cluster
            elif all([params.adult[0] <= age <= params.adult[-1] for age in cluster_age]):
                isolateid_to_agecategory[isolateid] = 'A'
                for isolateid in cluster:
                    isolateid_to_representatives[isolateid] = cluster
            # uncategorized age present in sequence cluster 
            elif all([params.child[0] <= age <= params.adult[-1] for age in cluster_age]) and any([params.child[-1] < age < params.adult[0] for age in cluster_age]):
                isolateid_to_agecategory[isolateid] = 'U'
                for isolateid in cluster:
                    isolateid_to_representatives[isolateid] = [member for _, member in enumerate(cluster) if params.child[-1] < cluster_age[_] < params.adult[0]]
            else:
                isolates_to_ignore = list(set(isolates_to_ignore)|set(cluster))

# print example
example_isolateid = random.choice(isolateid_to_agecategory.keys())
print ('EXAMPLE of the age category of a CD-HIT cluster:\nRepresentative sequence = {}'.format(isolateid_to_fdatheader[example_isolateid]))
print ('Age category: {}'.format(isolateid_to_agecategory[example_isolateid]))
print ('Member sequences:\n{}'.format('\n'.join([isolateid_to_fdatheader[_] for _ in isolateid_to_representatives[example_isolateid]])))

EXAMPLE of the age category of a CD-HIT cluster:
Representative sequence = A/SouthCarolina/22/2016_HA_EPIISL225062_2016.39767283_ORI/CLI_A/H3N2_LAT33.836081LON-81.163724_AGE18
Age category: U
Member sequences:
A/SouthCarolina/19/2016_HA_EPIISL225061_2016.39493498_ORI/CLI_A/H3N2_LAT33.836081LON-81.163724_AGE19
A/SouthCarolina/25/2016_HA_EPIISL225067_2016.39767283_ORI/CLI_A/H3N2_LAT33.836081LON-81.163724_AGE19
A/SouthCarolina/17/2016_HA_EPIISL225128_2016.39219713_ORI/CLI_A/H3N2_LAT33.836081LON-81.163724_AGE25
A/SouthCarolina/22/2016_HA_EPIISL225062_2016.39767283_ORI/CLI_A/H3N2_LAT33.836081LON-81.163724_AGE18


### Parsing phylogenetic tree using ete3

In [4]:
try:
    tree = ete3.Tree(params.tree)
except:
    sys.exit('\nERROR: Unable to parse tree using ete3.\n')
tree.ladderize()
# get nodes to isolate ids
leaf_to_isolateid = {leaf:re.search('(EPI_ISL_|EPIISL|GB_ISL_|GBISL)\d+', leaf.get_leaf_names()[0]).group().replace('_', '') for leaf in tree.get_leaves()}
isolateid_to_leaf = {v:k for k,v in leaf_to_isolateid.items()}

### Parsing for evolutionarily closest pairs 

To maximally identify virus pairs with differences while maintaining the evolutionary relatedness between these sequences at the closest level, note that all inferred patrsitic distances were truncated to 4 decimal places to disregard differences attributed to transitions/transversions.

Recall that the ML tree was rooted to maximally correlate root-to-tip distance with time. Starting from the root, we traverse the tree by level-order.  

1. For each internal node, we parse for the progeny leaf (tip) closest to the node, with the least number of unknown residues, as the ancestral sequence(s). Note that there CAN be >1 ancestral sequence.
2. Gather the rest of the progeny leaves as well as the grand-children leaves of the current internal node. 
3. Order the progeny and grand-children leaves by their patristic distance to node. 
4. Combinatorially, pair each of the progeny/grand-children leaf to the ancestral sequence(s) up to the stipulated order (in this case params.order = 3).

Note that certain sequences/pairs are omitted if: 
1. the sequence is part of the CD-HIT identical nucleotide sequence cluster age-categorized as [U]ncategorized 
2. the sequence pair resulted in >params.maxmut (5 in this case) amino acid changes. 
3. the patristic distance between the paired sequences is >params.patdist (0.007 in this case). 

In [5]:
anc_to_desc = {}
desc_to_anc = {}
pair_to_subinfo = {}
pair_to_pwdist = {}
# traverse through tree level-order
for node in tree.traverse():
    if node.is_leaf():
        continue

    # get children leaf nodes and their distances to current node
    childleaf_to_distance = {child:Decimal(child.get_distance(node)).quantize(Decimal('1e-4')) for child in node.children if child.is_leaf()}

    if len(childleaf_to_distance) > 0:
        # closest child leaf(s) from node - decide closest child leaf using patristic distances without truncating at 1e-5
        nearest_childleaf = [child for child in childleaf_to_distance.keys() if childleaf_to_distance[child] == min(childleaf_to_distance.values())]
        # remove sequences in cases where distance = 0 and aren't the least number of unknown residues
        if min(childleaf_to_distance.values()) == 0 and len(nearest_childleaf) > 1:
            nearest_childleaf, childleaf_to_distance = get_least_unknown_residue_sequences({child:fdat_nuc[isolateid_to_fdatheader[leaf_to_isolateid[child]]] for child in nearest_childleaf}, dist_dict_to_edit=childleaf_to_distance)

        # define nearest child leaf/leaves
        nearest_childleaf_distance_to_node = {child:childleaf_to_distance[child] for child in nearest_childleaf}
        for child in nearest_childleaf:
            del childleaf_to_distance[child]

        # get (great0-)^n-grandchildren leaves
        descendant_internal_nodes = [child for child in node.get_descendants() if not child.is_leaf() and node.get_distance(child) < params.patdist]
        if len(descendant_internal_nodes) > 0:
            for childnode in descendant_internal_nodes:
                grandchildren_leaves = [grandchild for grandchild in childnode.children if grandchild.is_leaf()]
                if len(grandchildren_leaves) > 0:
                    grandchildleaf_to_distance = {grandchild:Decimal(grandchild.get_distance(childnode)).quantize(Decimal('1e-4')) for grandchild in grandchildren_leaves}
                    nearest_grandchild_with_zero_distance = [grandchild for grandchild in grandchildren_leaves if grandchildleaf_to_distance[grandchild] == 0]
                    # remove sequences in cases where distance = 0 and aren't the least number of unknown residues
                    # we do this because we don't want to re-count possible identical strains
                    if len(nearest_grandchild_with_zero_distance) > 1:
                        nearest_grandchild_with_zero_distance_to_keep = get_least_unknown_residue_sequences({grandchild:fdat_nuc[isolateid_to_fdatheader[leaf_to_isolateid[grandchild]]] for grandchild in nearest_grandchild_with_zero_distance})
                        for grandchild in list(set(nearest_grandchild_with_zero_distance)-set(nearest_grandchild_with_zero_distance_to_keep)):
                            grandchildren_leaves.remove(grandchild)
                    # update grandchild distance to node
                    childleaf_to_distance.update({grandchild:Decimal(grandchild.get_distance(node)).quantize(Decimal('1e-4')) for grandchild in grandchildren_leaves})

        # if there are no children/grandchildren left for pairing
        if len(childleaf_to_distance) == 0:
            continue

        for anc_leaf in nearest_childleaf:
            # ignore ambiguous age isolates
            anc = leaf_to_isolateid[anc_leaf]
            if anc in isolates_to_ignore:
                continue

            # find closest pairing descendants
            # get pairwise distance to ancestor
            childleaf_to_pwdistance = {}
            for child, distance in childleaf_to_distance.items():
                pwdistance = nearest_childleaf_distance_to_node[anc_leaf] + distance
                childleaf_to_pwdistance[child] = pwdistance
                pair_to_pwdist[(anc, leaf_to_isolateid[child])] = pwdistance

            # analyze up to the n-th order set of pairs
            for pwdistance in sorted(set(childleaf_to_pwdistance.values()))[:params.order]:
                # patristic distance threshold
                if pwdistance > params.patdist:
                    continue

                children_with_distance = [child for child, child_distance in childleaf_to_pwdistance.items() if child_distance == pwdistance]
                for desc_leaf in children_with_distance:

                    desc = leaf_to_isolateid[desc_leaf]
                    if desc in isolates_to_ignore:
                        continue

                    # descendant should be further away from node
                    if childleaf_to_distance[desc_leaf] <= nearest_childleaf_distance_to_node[anc_leaf]:
                        continue

                    substitutions = get_pairwise_substitutions(fdat_aa[isolateid_to_fdatheader[anc]], fdat_aa[isolateid_to_fdatheader[desc]])
                    # amino acid substitutions threshold
                    if len(substitutions) > params.maxmut:
                        continue

                    # check that there is no nearer ancestor to descendant
                    if desc in desc_to_anc:
                        prev_anc_list = desc_to_anc[desc]
                        prev_anc_to_pwdist = {prev_anc:pair_to_pwdist[(prev_anc, desc)] for prev_anc in prev_anc_list}
                        prev_anc_to_pwdist[anc] = pwdistance
                        minimum_distance = min(prev_anc_to_pwdist.values())

                        for prev_anc, prev_anc_pwdist in prev_anc_to_pwdist.items():
                            if prev_anc_pwdist == minimum_distance:
                                if prev_anc in anc_to_desc:
                                    if desc not in anc_to_desc[prev_anc]:
                                        anc_to_desc[prev_anc].append(desc)
                                else:
                                    anc_to_desc[prev_anc] = [desc]

                                if desc in desc_to_anc:
                                    if prev_anc not in desc_to_anc[desc]:
                                        desc_to_anc[desc].append(prev_anc)
                                else:
                                    desc_to_anc[desc] = [prev_anc]

                                if (prev_anc, desc) not in pair_to_subinfo:
                                    pair_to_subinfo[(prev_anc, desc)] = (substitutions, get_pairwise_substitutions(fdat_nuc[isolateid_to_fdatheader[anc]], fdat_nuc[isolateid_to_fdatheader[desc]]))
                            else:
                                try:
                                    anc_to_desc[prev_anc].remove(desc)
                                    if len(anc_to_desc[prev_anc]) == 0:
                                        del anc_to_desc[prev_anc]
                                except:
                                    pass

                                try:
                                    desc_to_anc[desc].remove(prev_anc)
                                    if len(desc_to_anc[desc]) == 0:
                                        del desc_to_anc[desc]
                                except:
                                    pass
                    else:
                        try:
                            anc_to_desc[anc].append(desc)
                        except:
                            anc_to_desc[anc] = [desc]

                        try:
                            desc_to_anc[desc].append(anc)
                        except:
                            desc_to_anc[desc] = [anc]

                        pair_to_subinfo[(anc, desc)] = (substitutions, get_pairwise_substitutions(fdat_nuc[isolateid_to_fdatheader[anc]], fdat_nuc[isolateid_to_fdatheader[desc]]))

### Printing output 

Evolutionarily closest pairs are printed as a tab-delimited output (found in the [Files](https://github.com/alvinxhan/ageflu/tree/master/files) folder as ageflu_evol-closest-pairs.0C5_35A120_PD0.007_MM5_CL3_H3N2-HA-nuc.rooted.nwk.txt under [H3N2](https://github.com/alvinxhan/ageflu/tree/master/files/H3N2)): 

In [7]:
# get reference numbering 
ref_subtype_dictionary = {'H1N1PDM09':'H1', 'H3N2':'H3', 'BVIC':'B73', 'BYAM':'B73'}
HA_ref_numbering_fdat = parsefasta('./files/H1pdm09_H3_FluB_NumberingRef.fa', check_dna=0, check_codon=0)
queryST_to_refST_to_AbNum_to_RefNum = {'H1N1PDM09': {'H1':{}, 'H3':{}, 'B73':{}}, 'H3N2': {'H1':{}, 'H3':{}, 'B73':{}}, 'BVIC': {'H1':{}, 'H3':{}, 'B73':{}}, 'BYAM': {'H1':{}, 'H3':{}, 'B73':{}}}
queryST_to_RefStrain = {'H1N1PDM09':'A/Texas/04/2009(H1N1)_H1pdm09absolutenumbering', 'H3N2':'A/Aichi/2/1968(H3N2)_H3absolutenumbering', 'BVIC':'B/Brisbane/60/2008_BVicAbs', 'BYAM':'B/Phuket/3073/2013_BYamAbs'}
RefStrain_to_RefSeq = {'H1':HA_ref_numbering_fdat['A/Texas/04/2009(H1N1)_H1pdm09absolutenumbering'], 'H3':HA_ref_numbering_fdat['A/Aichi/2/68(H3N2)_H3numbering'], 'B73':HA_ref_numbering_fdat['B/HK/8/73_BYamNumbering']}
FluB_insertion_querypos_to_refpos = {177:162.1, 178:162.2, 179:162.3}

for subtype in queryST_to_RefStrain.keys():
    for RefStrain, RefSeq in RefStrain_to_RefSeq.items():
        CurrRefPosNum, CurrQueryPosNum = 0, 0
        for _ in xrange(len(RefSeq)):
            RecordPosBIN = -1
            if RefSeq[_] != '-':
                CurrRefPosNum += 1
                RecordPosBIN += 1
            if HA_ref_numbering_fdat[queryST_to_RefStrain[subtype]][_] != '-':
                CurrQueryPosNum += 1
                RecordPosBIN += 1
            if RecordPosBIN > 0:
                queryST_to_refST_to_AbNum_to_RefNum[subtype][RefStrain][CurrQueryPosNum] = CurrRefPosNum
            #162-163 insertion for Flu B (162.1, 162.2, 162.3)
            if (subtype == 'BVIC' or subtype == 'BYAM') and _ in range(177,180):
                if HA_ref_numbering_fdat[queryST_to_RefStrain[subtype]][_] != '-':
                    queryST_to_refST_to_AbNum_to_RefNum[subtype][RefStrain][CurrQueryPosNum] = FluB_insertion_querypos_to_refpos[_]

# print output 
with open('./files/{}/ageflu_evol-closest-pairs.{}.txt'.format(subtype_to_analyse, outfname), 'w') as output:
    output.write('pair_type\tanc\tdesc\tmutation(ABS)\tmutation(REF)\n')
    for anc, desclist in anc_to_desc.items():
        for desc in desclist:

            # determine pair type
            if params.child[0] <= isolateid_to_age[anc] <= params.child[-1]:
                anc_age_type = 'C'
            elif params.adult[0] <= isolateid_to_age[anc] <= params.adult[-1]:
                anc_age_type = 'A'
            else:
                anc_age_type = 'N'

            if params.child[0] <= isolateid_to_age[desc] <= params.child[-1]:
                desc_age_type = 'C'
            elif params.adult[0] <= isolateid_to_age[desc] <= params.adult[-1]:
                desc_age_type = 'A'
            else:
                desc_age_type = 'N'

            if anc_age_type == 'N' or desc_age_type == 'N':
                pair_age_type = 'NC'
            else:
                pair_age_type = '{}{}'.format(anc_age_type, desc_age_type)

            # get substitution info
            substitutions, nuc_sub_info = pair_to_subinfo[(anc, desc)]
            substitutions_abs_positions = [int(re.search('\d+', _).group()) for _ in substitutions]

            # get reference positions
            substitutions_ref_positions = [queryST_to_refST_to_AbNum_to_RefNum[query_subtype][ref_subtype_dictionary[query_subtype]][abs_pos] if abs_pos in queryST_to_refST_to_AbNum_to_RefNum[query_subtype][ref_subtype_dictionary[query_subtype]] else '-' for abs_pos in substitutions_abs_positions]

            # write to output
            write_line = map(str, [pair_age_type, isolateid_to_fdatheader[anc], isolateid_to_fdatheader[desc], ','.join(substitutions), ','.join(map(str, substitutions_ref_positions))])
            output.write('{}\n'.format('\t'.join(write_line)))
            output.flush()

## Age-association analyses 

With the evolutionarily closest pairs of sequences, we now perform the following age-association analyses. You can do so using [ageflu_analysis.py](https://github.com/alvinxhan/ageflu/blob/master/scripts/ageflu_analysis.py). Here, we show the breakdown of the script.

### Importing modules and defining functions

In [8]:
from __future__ import division
from scipy import stats
from math import sqrt
import statsmodels.api as sm
import subprocess, numpy

# check for N-X-S/T glycosylation motif changes
def check_glyco(positions, anc_seq, desc_seq):
    glyco_sites = []
    for pos in positions:
        if anc_seq[pos-1] == 'N' and re.search('N[^PX-][ST]', anc_seq[pos-1:pos+2]) and not re.search('N[^PX-][ST]', desc_seq[pos-1:pos+2]):
            glyco_sites.append(pos)
        elif re.search('[^PX-]', anc_seq[pos-1]) and re.search('N[^PX-][ST]', anc_seq[pos-2:pos+1]) and not re.search('N[^PX-][ST]', desc_seq[pos-2:pos+1]):
            glyco_sites.append(pos)
        elif re.search('[ST]', anc_seq[pos-1]) and re.search('N[^PX-][ST]', anc_seq[pos-3:pos]) and not re.search('N[^PX-][ST]', desc_seq[pos-3:pos]):
            glyco_sites.append(pos)

    return glyco_sites

# odds ratio analyses using R (called using subprocess)
def calculateOR(n11, n10, n01, n00):

    ORnum = n11*n00
    ORden = n01*n10

    if ORnum == ORden == 0:
        OR_uMLE, wald_MLE_CI95L, wald_MLE_CI95R = 'NaN', '', ''
        wald_MLE_CI90L, wald_MLE_CI90R = '', ''
    elif ORnum == 0:
        OR_uMLE, wald_MLE_CI95L, wald_MLE_CI95R = 0, '', ''
        wald_MLE_CI90L, wald_MLE_CI90R = '', ''
    elif ORden == 0:
        OR_uMLE, wald_MLE_CI95L, wald_MLE_CI95R = 'InF', '', ''
        wald_MLE_CI90L, wald_MLE_CI90R = '', ''
    else:
        OR_uMLE = ORnum/ORden
        # Wald test & CI
        lnOR = numpy.log(OR_uMLE)
        SE_lnOR = numpy.sqrt(sum(map(lambda _:1/_ ,[n11,n10,n01,n00])))
        wald_MLE_CI95L, wald_MLE_CI95R =  numpy.exp(lnOR - 1.96*SE_lnOR), numpy.exp(lnOR + 1.96*SE_lnOR)
        wald_MLE_CI90L, wald_MLE_CI90R =  numpy.exp(lnOR - 1.645*SE_lnOR), numpy.exp(lnOR + 1.645*SE_lnOR)

    if n11 + n10 + n01 + n00 >= 1000:
        # G-test
        cmd = ['Rscript', "./scripts/ageflu_gtest.R"] + map(str,[n11, n10, n01, n00])
        Routput = subprocess.check_output(cmd, universal_newlines=True, stderr=subprocess.PIPE).split('\n')
        pval = float(Routput[1])
        # small sample size CI - NaN
        ss_CI90L, ss_CI90R, ss_CI95L, ss_CI95R = 'NaN', 'NaN', 'NaN', 'NaN'
    else:
        # Barnard's
        cmd = ['Rscript', "./scripts/ageflu_barnard.R"] + map(str,[n11, n10, n01, n00])
        Routput = subprocess.check_output(cmd, universal_newlines=True, stderr=subprocess.PIPE).split('\n')
        pval = float(Routput[-2].replace('[1]','').split()[1])
        # Agresti & Min unconditional exact CI for small sample sizes
        cmd = ['Rscript', "./scripts/agresti_and_min.R"] + map(str,[n11, n10, n01, n00, 0.9])
        Routput = filter(None, subprocess.check_output(cmd, universal_newlines=True, stderr=subprocess.PIPE).split('\n'))
        ss_CI90L, ss_CI90R = map(lambda _: _ if _ == 'Inf' else float(_), re.findall('(\d+\.\d+|\d+|Inf)', Routput[-1]))

        cmd = ['Rscript', "./scripts/agresti_and_min.R"] + map(str,[n11, n10, n01, n00, 0.95])
        Routput = filter(None, subprocess.check_output(cmd, universal_newlines=True, stderr=subprocess.PIPE).split('\n'))
        ss_CI95L, ss_CI95R = map(lambda _: _ if _ == 'Inf' else float(_), re.findall('(\d+\.\d+|\d+|Inf)', Routput[-1]))

    return OR_uMLE, wald_MLE_CI90L, wald_MLE_CI90R, wald_MLE_CI95L, wald_MLE_CI95R, pval, ss_CI90L, ss_CI90R, ss_CI95L, ss_CI95R

### Define inputs 

In [7]:
if passage_requirement == 'F':
    class params():
        input_file = './files/{st}/ageflu_evol-closest-pairs.0C{child_max_age}_{adult_min_age}A120_PD0.007_MM5_CL3_{st}-HA-nuc_rooted.nwk.txt'.format(st=subtype_to_analyse, 
                                                                                                                                                      child_max_age=subtype_to_passage_to_age_range[passage_requirement][subtype_to_analyse]['child'][-1],
                                                                                                                                                      adult_min_age=subtype_to_passage_to_age_range[passage_requirement][subtype_to_analyse]['adult'][0]) # ageflu_getpairs.py output
        aln = './files/{st}/{st}-HA-nuc.fa'.format(st=subtype_to_analyse) # Nucleotide alignment of HA sequences (pre-CD-HIT)
        same_passage = False # binary whether to implement filter to only analyse sequence pairs derived from the same passage type 
else:
    class params():
        input_file = './files/{st}/ageflu_evol-closest-pairs.0C{child_max_age}_{adult_min_age}A120_PD0.007_MM5_CL3_{st}-HA-nuc_rooted.nwk.txt'.format(st=subtype_to_analyse, 
                                                                                                                                                      child_max_age=subtype_to_passage_to_age_range[passage_requirement][subtype_to_analyse]['child'][-1],
                                                                                                                                                      adult_min_age=subtype_to_passage_to_age_range[passage_requirement][subtype_to_analyse]['adult'][0]) # ageflu_getpairs.py output
        aln = './files/{st}/{st}-HA-nuc.fa'.format(st=subtype_to_analyse) # Nucleotide alignment of HA sequences (pre-CD-HIT)
        same_passage = True #

./files/H3N2/ageflu_evol-closest-pairs.0C5_35A120_PD0.007_MM5_CL3_H3N2-HA-nuc_rooted.nwk.txt


### Define parameters - Known canonical antigenic/receptor-binding sites + reference positions



In [10]:
### POSITION PARAMETERS - FluA (all based on H3 numbering)###
# antigenic sites gathered from Stray, S. J., & Pittman, L. B. (2012). Subtype-and antigenic site-specific differences in biophysical influences on evolution of influenza virus hemagglutinin. Virology journal, 9(1), 1.
H3ant = {"A": range(121,130) + range(131,139) + range(142,147)  + [140],
         "B": range(155,161) + range(188,191) + range(192,195) + range(196,200) + [186, 246, 247],
         "C": [49, 50, 53, 54, 271, 273, 275, 276, 278],
         "D": range(201,208) + range(216, 221) + range(225, 228) + [167, 214, 222, 223, 242],
         "E": range(79, 84) + [62, 63, 75, 78, 91, 92]}

H1ant = {"Sa": range(165,168) + [128, 129, 163, 247, 248],
         "Sb": [156, 159, 189, 190, 192, 193, 196, 197, 198],
         "Ca1": range(240, 246) + [169, 173, 207, 212],
         "Ca2": [132, 140, 143, 144, 145, 149, 224, 225],
         "Cb": range(259, 266) + [56, 79, 80, 82, 83, 117, 149, 255, 256],
         "H1C": range(271, 277) + range(278, 282) + [90, 285]}

# H3 rbs gathered from Yang, H., Carney, P. J., Chang, J. C., Guo, Z., Villanueva, J. M., & Stevens, J. (2015). Structure and receptor binding preferences of recombinant human A (H3N2) virus hemagglutinins. Virology, 477, 18-31 - based on H3 numbering
H3RBS = [98, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 144, 145, 146, 153, 154, 155, 156, 157, 158, 159, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196] + range(219,229)

#passage sites - based on H3 numbering
weiwei22pas = [111, 126, 137, 138, 144, 145, 155, 156, 158, 159, 185, 186, 193, 194, 199, 219, 226, 229, 246, 248, 276, 290]
RaphaelPas = [13, 96, 137, 156, 183, 186, 189, 190, 193, 194, 196, 203, 219, 222, 246, 248, 328, 332, 335, 337, 352, 353, 376, 447, 465, 504, 515]
SebastianPas = [101, 103, 123, 129, 131, 133, 137, 140, 142, 144, 145, 156, 157, 158, 159, 165, 187, 188, 192, 193, 196, 197, 198, 218, 222, 225, 226, 227, 252, 262, 269, 273, 279, 312, 474, 476, 498, 45, 510, 529, 48]

### POSITION PARAMETERS - Flu B (all based on B/HK/8/73 numbering)###
FluB_insertion_querypos_to_refpos = {177:162.1, 178:162.2, 179:162.3}
FluBant = {"120loop":range(51,55) + range(56,60)+range(73,76)+range(121,124)+range(179,182)+range(283,286)+[129,137],
           "150loop":range(139,143) + range(145,152),
           "160loop":range(162,166) + range(167,172),
           "190helix":range(194,201) + range(235,241)+[205],
           "162-163insertion":[162.1, 162.2, 162.3]}
FluBRBS = range(193,203) + range(237,243) + range(136,144) + [95, 158, 191, 202]


if query_subtype == 'H3N2':
    AS_dict, RBS_list = H3ant, H3RBS
elif query_subtype == 'H1N1PDM09':
    AS_dict, RBS_list = H1ant, H3RBS
else:
    AS_dict, RBS_list = FluBant, FluBRBS
# AS + RBS
functional_sites = list(set(RBS_list)|set([k for v in AS_dict.values() for k in v]))

### Parse input files and count pairs/substitution frequencies 

We first parse the input files and count the number of pairs for each pair-type and their corresponding amino acid substitution frequencies.

In [11]:
# read fasta file
fdat_nuc = parsefasta(params.aln)
fdat_aa = {k:translatedDNA(v) for k,v in fdat_nuc.items()}
header_to_passage = {header:re.search('(ORI|MDCK|SIAT|EGG)', header).group() for header in fdat_nuc.keys()}

# read input file and count pairs/substitutions
pt_count, pt_to_AAsublen, pt_to_RBASsub, abspos_to_pt_to_count, abspos_to_mutation_to_pt_to_count, abspos_to_refpos = {}, {}, {}, {}, {}, {}
all_potential_glycosylation_sites = []

fhandle = filter(None, open(params.input_file, 'rU'))
fhandle.pop(0)
for line in fhandle:
    try:
        pt, anc, desc, substitutions, substitutions_ref_positions = line.strip().split('\t')
        substitutions = substitutions.split(',')
    except:
        pt, anc, desc = line.strip().split('\t')
        substitutions = []

    # same passage filter
    if params.same_passage and header_to_passage[anc] != header_to_passage[desc]:
        continue

    # add to pair type count
    try:
        pt_count[pt] += 1
    except:
        pt_count[pt] = 1

    # add to pair type substitution count
    try:
        pt_to_AAsublen[pt][len(substitutions)] += 1
    except:
        try:
            pt_to_AAsublen[pt][len(substitutions)] = 1
        except:
            pt_to_AAsublen[pt] = {len(substitutions):1}

    # continue if NC pairs
    if pt == 'NC':
        continue

    if len(substitutions) > 0:
        # check if any substitutions are potential glycosylation sites
        substitutions_abs_positions = [int(re.search('\d+', _).group()) for _ in substitutions]
        potential_glycosylation_sites = check_glyco(substitutions_abs_positions, fdat_aa[anc], fdat_aa[desc])
        all_potential_glycosylation_sites = list(set(all_potential_glycosylation_sites)|set(potential_glycosylation_sites))


        for p, abs_pos in enumerate(substitutions_abs_positions):
            # add to position substitution count
            try:
                abspos_to_pt_to_count[abs_pos][pt] += 1
            except:
                try:
                    abspos_to_pt_to_count[abs_pos][pt] = 1
                except:
                    abspos_to_pt_to_count[abs_pos] = {pt:1}

            # add to pos-mutation dictionary
            substitution = substitutions[p]
            try:
                abspos_to_mutation_to_pt_to_count[abs_pos][substitution][pt] += 1
            except:
                try:
                    abspos_to_mutation_to_pt_to_count[abs_pos][substitution][pt] = 1
                except:
                    try:
                        abspos_to_mutation_to_pt_to_count[abs_pos][substitution] = {pt:1}
                    except:
                        abspos_to_mutation_to_pt_to_count[abs_pos] = {substitution:{pt:1}}

        # count if any sites are either RBS/AS
        ref_positions = []
        for _ in substitutions_ref_positions.split(','):
            if _ == '-':
                ref_positions.append('-')
            elif re.search('162(\.1|\.2|\.3)', _):
                ref_positions.append(float(_))
            else:
                ref_positions.append(int(_))
        for _, refpos in enumerate(ref_positions):
            abspos_to_refpos[substitutions_abs_positions[_]] = refpos

        if set(functional_sites)&set(ref_positions):
            try:
                pt_to_RBASsub[pt] += 1
            except:
                pt_to_RBASsub[pt] = 1


## Pairs distribution analyses

### Print distribution of pairs stratified by number/proportion of amino acid substitutions

We generate the output file (in this case, ageflu_pairs-distribution.0C5_35A120_PD0.007_MM5_CL3_H3N2-HA-nuc_rooted.nwk_SP0.txt). First, we print the distribution of pairs stratified by the number/proportion of amino acid substitutions.

In [12]:
maxmut = int(re.search('_MM(\d+)_', params.input_file).group(1))
outfname = '{}_SP{}.txt'.format(re.sub('/*[^/]+/', '', params.input_file.replace('ageflu_evol-closest-pairs.', '').replace('.txt', '')), '1' if params.same_passage else '0')

# write to output
output = open('ageflu_pairs-distribution.{}'.format(outfname), 'w')

# pair distribution - frequencies
output.write('Frequency distribution of pair types stratified by number of amino acid substitutions\nPT/Sub_#\tCA\tAC\tCC\tAA\tNC\tSub#_TOT\n')
col_counts = [0]*(maxmut)
sublen_to_row_counts = {}
for sub_len in xrange(maxmut+1):
    try:
        CA_count = pt_to_AAsublen['CA'][sub_len]
    except:
        CA_count = 0
        try:
            pt_to_AAsublen['CA'][sub_len] = 0
        except:
            pt_to_AAsublen['CA']={sub_len:0}

    try:
        AC_count = pt_to_AAsublen['AC'][sub_len]
    except:
        AC_count = 0
        try:
            pt_to_AAsublen['AC'][sub_len] = 0
        except:
            pt_to_AAsublen['AC'] = {sub_len: 0}

    try:
        CC_count = pt_to_AAsublen['CC'][sub_len]
    except:
        CC_count = 0
        try:
            pt_to_AAsublen['CC'][sub_len] = 0
        except:
            pt_to_AAsublen['CC'] = {sub_len: 0}

    try:
        AA_count = pt_to_AAsublen['AA'][sub_len]
    except:
        AA_count = 0
        try:
            pt_to_AAsublen['AA'][sub_len] = 0
        except:
            pt_to_AAsublen['AA'] = {sub_len: 0}

    try:
        NC_count = pt_to_AAsublen['NC'][sub_len]
    except:
        NC_count = 0
        try:
            pt_to_AAsublen['NC'][sub_len] = 0
        except:
            pt_to_AAsublen['NC'] = {sub_len: 0}

    row_counts = [CA_count, AC_count, CC_count, AA_count, NC_count]
    sublen_to_row_counts[sub_len] = row_counts

    for k, count in enumerate(row_counts):
        col_counts[k] += count
    output.write('{}\t{}\t{}\n'.format(sub_len, '\t'.join(map(str, row_counts)), sum(row_counts)))
output.write('PT_TOT\t{}\t\n\n'.format('\t'.join(map(str, col_counts))))

# pairs distribution - proportions
output.write('Distribution of pair types stratified by number of amino acid substitutions\nPT/Sub_#\tCA\tAC\tCC\tAA\tNC\n')
for sub_len in xrange(maxmut+1):
    div_list = [ ]
    for k, col_tot in enumerate(col_counts):
        try:
            div_list.append(sublen_to_row_counts[sub_len][k]/col_tot)
        except:
            div_list.append('NaN')

    output.write('{}\t{}\n'.format(sub_len, '\t'.join(map(lambda _: '{}'.format(_), div_list))))
output.write('\n')

### Association analyses between pair types (end-in-adult v. end-in-child) and observation for substitution(s) - whole HA protein

Next, we perform two-way contingency table analyses between pair types (end-in-adult (CA/AA) pairs vs. end-in-child (CC/AC) pairs) and if changes between the paired sequences were observed. Here, we consider substitutions across the entire HA protein.

In [13]:
try:
    n1t = pt_count['CA']
except:
    n1t = 0

try:
    n1t += pt_count['AA']
except:
    pass

try:
    n0t = pt_count['CC']
except:
    n0t = 0

try:
    n0t += pt_count['AC']
except:
    pass

try:
    n11 = sum([pt_to_AAsublen['CA'][_] for _ in pt_to_AAsublen['CA'] if _ > 0])
except:
    n11 = 0

try:
    n11 += sum([pt_to_AAsublen['AA'][_] for _ in pt_to_AAsublen['AA'] if _ > 0])
except:
    pass

try:
    n01 = sum([pt_to_AAsublen['CC'][_] for _ in pt_to_AAsublen['CC'] if _ > 0])
except:
    n01 = 0

try:
    n01 += sum([pt_to_AAsublen['AC'][_] for _ in pt_to_AAsublen['AC'] if _ > 0])
except:
    pass

output.write('Association analyses between pair-type (end-in-adult v. end-in-child) and propensity for substitution(s) - whole HA\nPT/SF\tSubstitution\tNo Substitution\nEnd-in-Adult\t{}\t{}\nEnd-in-Child\t{}\t{}\n'.format(n11, n1t-n11, n01, n0t-n01))
OR_uMLE, wald_MLE_CI90L, wald_MLE_CI90R, wald_MLE_CI95L, wald_MLE_CI95R, pval, ss_CI90L, ss_CI90R, ss_CI95L, ss_CI95R = calculateOR(n11, n1t-n11, n01, n0t-n01)
output.write('OR\t{}\tp-value\t{}\nWald_95CI\t{}\nWald_90CI\t{}\nA&M_90CI\t{}\nA&M_95CI\t{}\n\n'.format(OR_uMLE, pval, ','.join(map(lambda _: '{}'.format(_), [wald_MLE_CI90L, wald_MLE_CI90R])), ','.join(map(lambda _: '{}'.format(_), [wald_MLE_CI95L, wald_MLE_CI95R])), ','.join(map(lambda _: '{}'.format(_), [ss_CI90L, ss_CI90R])), ','.join(map(lambda _: '{}'.format(_), [ss_CI95L, ss_CI95R]))))

### Association analyses between pair types (end-in-adult v. end-in-child) and observation for substitution(s) - RBS/AS only

Next, we perform the same association analysis but only consider substitutions within the receptor-binding/antigenic regions only:

In [14]:
# n1t and n0t stays the same
try:
    n11 = pt_to_RBASsub['CA']
except:
    n11 = 0

try:
    n11 += pt_to_RBASsub['AA']
except:
    pass

try:
    n01 = pt_to_RBASsub['CC']
except:
    n01 = 0

try:
    n01 += pt_to_RBASsub['AC']
except:
    pass

output.write('Association analyses between pair-type (end-in-adult v. end-in-child) and propensity for substitution(s) - RBS/AS\nPT/SF\tSubstitution\tNo Substitution\nEnd-in-Adult\t{}\t{}\nEnd-in-Child\t{}\t{}\n'.format(n11, n1t-n11, n01, n0t-n01))
OR_uMLE, wald_MLE_CI90L, wald_MLE_CI90R, wald_MLE_CI95L, wald_MLE_CI95R, pval, ss_CI90L, ss_CI90R, ss_CI95L, ss_CI95R = calculateOR(n11, n1t-n11, n01, n0t-n01)
output.write('OR\t{}\tp-value\t{}\nWald_95CI\t{}\nWald_90CI\t{}\nA&M_90CI\t{}\nA&M_95CI\t{}\n\n'.format(OR_uMLE, pval, ','.join(map(lambda _: '{}'.format(_), [wald_MLE_CI90L, wald_MLE_CI90R])), ','.join(map(lambda _: '{}'.format(_), [wald_MLE_CI95L, wald_MLE_CI95R])), ','.join(map(lambda _: '{}'.format(_), [ss_CI90L, ss_CI90R])), ','.join(map(lambda _: '{}'.format(_), [ss_CI95L, ss_CI95R]))))

output.close()

### Determining sites associated with age

Finally, we perform association analyses to identify amino acid sites which substitution frequencies are associated with pair types (end-in-adult v. end-in-child). Here, we also identify if they are part of the receptor-binding or/and antigenic regions, as well as if they are potential glycosylation sites.

In [15]:
abspos_to_OR_results = {}
for abs_pos in abspos_to_pt_to_count.keys():
    # end-in-adult v. end-in-child
    try:
        n11 = abspos_to_pt_to_count[abs_pos]['CA']
        #CA_11 = abspos_to_pt_to_count[abs_pos]['CA']
    except:
        n11 = 0
        #CA_11 = 0

    try:
        n11 += abspos_to_pt_to_count[abs_pos]['AA']
    except:
        pass

    try:
        n01 = abspos_to_pt_to_count[abs_pos]['CC']
    except:
        n01 = 0

    try:
        n01 += abspos_to_pt_to_count[abs_pos]['AC']
        #AC_11 = abspos_to_pt_to_count[abs_pos]['AC']
    except:
        #AC_11 = 0
        pass

    n10 = pt_count['CA'] + pt_count['AA'] - n11
    n00 = pt_count['CC'] + pt_count['AC'] - n01

    abspos_to_OR_results[abs_pos] = {'all': calculateOR(n11, n10, n01, n00)}#, 'caac': calculateOR(CA_11, pt_count['CA']-CA_11, AC_11, pt_count['AC']-AC_11)}

# get q values
all_OR_pvals = [abspos_to_OR_results[abs_pos]['all'][5] for abs_pos in abspos_to_OR_results.keys()]
all_OR_qvals = sm.stats.multipletests(all_OR_pvals, method='fdr_bh')[1].tolist()
all_pval_to_qval = {all_OR_pvals[_]:qvalue for _, qvalue in enumerate(all_OR_qvals)}

with open('./files/{}/ageflu_age-associated-sites.{}'.format(subtype_to_analyse, outfname), 'w') as output:
    output.write('abs_pos\tref(H3)\tref(H1)\tref(B73)\tRBS\tAS\tGS\tPS\t'
                 'OR_uMLE\twald_MLE_CI90L\twald_MLE_CI90R\twald_MLE_CI95L\twald_MLE_CI95R\t'
                 'p-value\tss_CI90L\tss_CI90R\tss_CI95L\tss_CI95R\tq-value\t'
                 'Mutations({})\tAC\tCA\tAA\tCC\tEnd-in-Adult\tEnd-in-Child\tTotal\n'.format(ref_subtype_dictionary[query_subtype]))
    for abs_pos in abspos_to_OR_results.keys():
        # reference positions
        try:
            main_ref_pos = queryST_to_refST_to_AbNum_to_RefNum[query_subtype][ref_subtype_dictionary[query_subtype]][abs_pos]
        except:
            main_ref_pos = '-'
        try:
            H3_ref_pos = queryST_to_refST_to_AbNum_to_RefNum[query_subtype]['H3'][abs_pos]
        except:
            H3_ref_pos = '-'
        try:
            H1_ref_pos = queryST_to_refST_to_AbNum_to_RefNum[query_subtype]['H1'][abs_pos]
        except:
            H1_ref_pos = '-'
        try:
            B_ref_pos = queryST_to_refST_to_AbNum_to_RefNum[query_subtype]['B73'][abs_pos]
        except:
            B_ref_pos = '-'

        if re.search('(H1N1PDM09|H3N2)', query_subtype):
            RBS_binary = 1 if H3_ref_pos in RBS_list else ''
        else:
            RBS_binary = 1 if main_ref_pos in RBS_list else ''

        try:
            if re.search('(H1N1PDM09|H3N2)', query_subtype):
                AS_type = [_ for _ in AS_dict if H3_ref_pos in AS_dict[_]][0]
            else:
                AS_type = [_ for _ in AS_dict if main_ref_pos in AS_dict[_]][0]
        except:
            AS_type = ''
        GS_binary = 1 if abs_pos in all_potential_glycosylation_sites else ''
        PS_binary = 1 if H3_ref_pos in SebastianPas else ''

        # OR results
        all_OR_result = map(str, ['{:4f}'.format(_) if isinstance(_, float) else _ for _ in abspos_to_OR_results[abs_pos]['all']])
        #caac_OR_result = map(str, ['{:4f}'.format(_) if isinstance(_, float) else _ for _ in abspos_to_OR_results[abs_pos]['caac']])

        # substitutions
        substitution_line = ['{}({})'.format(re.sub(str(abs_pos), str(main_ref_pos), substitution), ','.join(['{}:{}'.format(pt, count) for pt, count in abspos_to_mutation_to_pt_to_count[abs_pos][substitution].items()])) for substitution in abspos_to_mutation_to_pt_to_count[abs_pos].keys()]

        # pair counts
        count_line = []
        end_in_adult_count, end_in_child_count = 0, 0
        try:
            count_line.append(abspos_to_pt_to_count[abs_pos]['AC'])
            end_in_adult_count += abspos_to_pt_to_count[abs_pos]['AC']
        except:
            count_line.append(0)

        try:
            count_line.append(abspos_to_pt_to_count[abs_pos]['CA'])
            end_in_child_count += abspos_to_pt_to_count[abs_pos]['CA']
        except:
            count_line.append(0)

        try:
            count_line.append(abspos_to_pt_to_count[abs_pos]['AA'])
            end_in_adult_count += abspos_to_pt_to_count[abs_pos]['AA']
        except:
            count_line.append(0)

        try:
            count_line.append(abspos_to_pt_to_count[abs_pos]['CC'])
            end_in_child_count += abspos_to_pt_to_count[abs_pos]['CC']
        except:
            count_line.append(0)

        count_line = count_line + [end_in_adult_count, end_in_child_count, end_in_adult_count+end_in_child_count]

        write_line = map(str, [abs_pos, H3_ref_pos, H1_ref_pos, B_ref_pos, RBS_binary, AS_type, GS_binary, PS_binary, '\t'.join(all_OR_result), all_pval_to_qval[abspos_to_OR_results[abs_pos]['all'][5]], ';'.join(substitution_line), '\t'.join(map(str, count_line))])

        output.write('{}\n'.format('\t'.join(write_line)))
        output.flush()